In [1]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           334Gi       2.1Gi       316Gi       2.0Mi        16Gi       330Gi
Swap:             0B          0B          0B


In [ ]:
# !pip install -q transformers==4.43.1 wandb

In [3]:
!pip uninstall -y torchvision

Found existing installation: torchvision 0.18.0+cpu
Uninstalling torchvision-0.18.0+cpu:
  Successfully uninstalled torchvision-0.18.0+cpu


In [2]:
%%capture
!pip install torch~=2.4.0 torch_xla[tpu]~=2.4.0 -f https://storage.googleapis.com/libtpu-releases/index.html
# !git clone -b llama2-google-next-training https://github.com/pytorch-tpu/transformers.git
%cd transformers
!pip install peft
!pip install -e . --user
!pip install datasets accelerate evaluate scikit-learn
!pip uninstall -y tensorflow jax

In [3]:
!git clone https://github.com/AlvinKimata/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 187217, done.
remote: Counting objects: 100% (22845/22845), done.
remote: Compressing objects: 100% (1815/1815), done.
remote: Total 187217 (delta 22243), reused 21067 (delta 21018), pack-reused 164372 (from 1)
Receiving objects: 100% (187217/187217), 206.40 MiB | 35.16 MiB/s, done.
Resolving deltas: 100% (134990/134990), done.


In [3]:
%cd /content/transformers

/content/transformers


In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [8]:
%cd transformers/examples/pytorch/language-modeling/

/content/transformers/transformers/examples/pytorch/language-modeling


In [38]:
import wandb
wandb.login(key = "", verify = True)


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [39]:
!export WANDB_PROJECT="law-llama-instruct"

In [ ]:
# Run
!python run_clm.py \
  --tokenizer_name hf-internal-testing/llama-tokenizer \
  --dataset_name CognifuseRL/kenyan_law_dataset \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --num_train_epochs 1 \
  --do_train \
  --do_eval \
  --evaluation_strategy "steps" \
  --eval_steps 100 \
  --output_dir /tmp/output \
  --overwrite_output_dir \
  --model_name_or_path "meta-llama/Meta-Llama-3.1-8B-Instruct"\
  --save_strategy "epoch" \
  --logging_strategy "steps" \
  --remove_unused_columns no \
  --optim adafactor \
  --torch_dtype bfloat16 \
  --dataloader_drop_last yes \
  --block_size 4096 \
  --spmd_2d_sharding 1 \
  --spmd_debug True \
  --spmd_grad_chkpt True \
  --peft_lora True \
  --trust_remote_code True \
  --push_to_hub True \
  --push_to_hub_model_id "law-llama-instruct" \
  --push_to_hub_token ""\
  --report_to 'wandb'

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:2007: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:2026: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Kimata/law-llama-instruct).
  warnings.warn(
INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumu

In [26]:
!ls -a /tmp/output/runs/Aug26_16-32-31_e7b57beda44b

.  ..  events.out.tfevents.1724689968.e7b57beda44b.31525.0


In [27]:
ls -a /tmp/output/

./  ../  runs/


In [29]:
!zip /content/run.zip /tmp/output/runs/Aug26_16-32-31_e7b57beda44b

  adding: tmp/output/runs/Aug26_16-32-31_e7b57beda44b/ (stored 0%)


In [30]:
!du -sh /tmp/output/runs/Aug26_16-32-31_e7b57beda44b

12K	/tmp/output/runs/Aug26_16-32-31_e7b57beda44b
